In [1]:
import xml.etree.ElementTree as ET
import os

In [2]:
def annotation_converter(xml_path, image_dir, output_dir):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    IMG_W = int(root.find("size/width").text)
    IMG_H = int(root.find("size/height").text)
    TILE_SIZE = 640
    os.makedirs(output_dir, exist_ok=True)

    image_names = set(img.split('.')[0] for img in os.listdir(image_dir))

    boxes = []
    for obj in root.findall("object"):
        if obj.find("name").text != "rumex":
            continue
        bbox = obj.find("bndbox")
        xmin = int(bbox.find("xmin").text)
        ymin = int(bbox.find("ymin").text)
        xmax = int(bbox.find("xmax").text)
        ymax = int(bbox.find("ymax").text)
        boxes.append((xmin, ymin, xmax, ymax))

    xml_name = os.path.basename(xml_path).split('.')[0]

    for y in range(0, IMG_H, TILE_SIZE):
        for x in range(0, IMG_W, TILE_SIZE):
            if y + TILE_SIZE > IMG_H or x + TILE_SIZE > IMG_W:
                continue

            tile_base = f"{xml_name}_x{x}_y{y}"
            if tile_base not in image_names:
                continue

            tile_boxes = []
            for xmin, ymin, xmax, ymax in boxes:
                if xmax < x or xmin > x + TILE_SIZE or ymax < y or ymin > y + TILE_SIZE:
                    continue

                x1 = max(xmin, x)
                y1 = max(ymin, y)
                x2 = min(xmax, x + TILE_SIZE)
                y2 = min(ymax, y + TILE_SIZE)

                cx = (x1 + x2) / 2 - x
                cy = (y1 + y2) / 2 - y
                w = x2 - x1
                h = y2 - y1

                tile_boxes.append((0, cx / TILE_SIZE, cy / TILE_SIZE, w / TILE_SIZE, h / TILE_SIZE))

            label_file = os.path.join(output_dir, tile_base + ".txt")
            with open(label_file, "w") as f:
                for box in tile_boxes:
                    f.write(f"{box[0]} " + " ".join(f"{val:.6f}" for val in box[1:]) + "\n")

    print(f"Annotation files saved in {output_dir}/")


In [3]:
xml_files = ['WENR_ortho_Rumex_10m_1_nw.xml', 'WENR_ortho_Rumex_10m_2_sw.xml', 'WENR_ortho_Rumex_10m_3_ne.xml', 
             'WENR_ortho_Rumex_10m_4_se.xml', 'WENR_ortho_Rumex_10m_2_sw.xml', 'WENR_ortho_Rumex_10m_4_se.xml']
output_dir = 'cocodronedata/labels/test'
image_path = 'cocodronedata/images/test'

for xml_file in xml_files:
    xml_path = f'dataset/{xml_file}'
    annotation_converter(xml_path, image_path, output_dir)

Annotation files saved in cocodronedata/labels/test/
Annotation files saved in cocodronedata/labels/test/
Annotation files saved in cocodronedata/labels/test/
Annotation files saved in cocodronedata/labels/test/
Annotation files saved in cocodronedata/labels/test/
Annotation files saved in cocodronedata/labels/test/


In [10]:
annotations = 0
for file in os.listdir(output_dir):
    if file.endswith('.txt'):
        with open(os.path.join(output_dir, file), 'r') as f:
            lines = f.readlines()
            annotations += len(lines)

print(f"Total annotation boxes: {annotations}")

Total annotation boxes: 657
